In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import tensorflow as tf
import cv2
import pandas as pd
import keras
import matplotlib.image as mpimg
!pip install visualkeras
import visualkeras

from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop


from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout,  Dense

from keras.layers import BatchNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# rootdir= '/content/drive/My Drive/hda/yeni_colab/Chest_Xray_Data'

# # the path of the dataset
# classes = ['covid','normal','pneumonia'] # the classes of the dataset

# for i in classes:
#     os.makedirs(rootdir + '/train/' + i)
#     os.makedirs(rootdir + '/validation/' + i)
#     os.makedirs(rootdir + '/test/' + i)

#     source = rootdir + '/' + i
#     allFileNames = os.listdir(source)
#     np.random.shuffle(allFileNames)

#     test_ratio = 0.2
#     validation_ratio = 0.25

#     FileNames, test_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames) * (1 - test_ratio))])
#     train_FileNames, validation_FileNames = np.split(np.array(FileNames), [int(len(FileNames) * (1 - validation_ratio))])

#     train_FileNames = [source + '/' + name for name in train_FileNames.tolist()]
#     validation_FileNames = [source + '/' + name for name in validation_FileNames.tolist()]
#     test_FileNames = [source + '/' + name for name in test_FileNames.tolist()]

#     for name in train_FileNames:
#         shutil.copy(name, rootdir + '/train/' + i)

#     for name in validation_FileNames:
#         shutil.copy(name, rootdir + '/validation/' + i)

#     for name in test_FileNames:
#         shutil.copy(name, rootdir + '/test/' + i)

In [ ]:
# Data Augmentation

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)




In [ ]:
# Training Data Generator


train_generator = train_datagen.flow_from_directory(directory=
    '/content/drive/My Drive/hda/yeni_colab/Chest_Xray_Data/train',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
# we dont use the data augmentation on testing and validation because:
# Applying data augmentation to the testing data would introduce additional
# variations that the model hasn't encountered during training, potentially leading to an overly optimistic or misleading evaluation.

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)
validation_generator=validation_datagen.flow_from_directory('/content/drive/My Drive/hda/yeni_colab/Chest_Xray_Data/validation',
                                        target_size=(128,128),
                                        batch_size=64,
                                        class_mode='categorical',
                                        shuffle=True
                                                         )

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)
test_generator=test_datagen.flow_from_directory('/content/drive/My Drive/hda/yeni_colab/Chest_Xray_Data/test',
                                        target_size=(128,128),
                                        batch_size=64,
                                        class_mode='categorical',
                                        shuffle=True
                                                         )

In [ ]:
train_generator.class_indices

# 1) CUSTOM MODEL with RMSprop




In [ ]:

model1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),



    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),



    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),

    tf.keras.layers.Dense(3, activation='softmax')
])

model1.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['categorical_accuracy'])


In [ ]:
visualkeras.layered_view(model1,legend=True)

In [ ]:
model1.summary()

In [ ]:
history1=model1.fit(train_generator, epochs=30, validation_data=validation_generator)

In [ ]:
model1.save('/content/drive/My Drive/hda/yeni_colab/models/model1')

np.save('/content/drive/My Drive/hda/Weights_model1.npy',history1.history)

In [ ]:
history1 = keras.models.load_model('/content/drive/My Drive/hda/yeni_colab/models/model1')


In [ ]:
history1.evaluate(train_generator)
history1.evaluate(validation_generator)
history1.evaluate(test_generator)

In [ ]:
from matplotlib import pyplot as plt

hist1=np.load('Weights_model1.npy',allow_pickle='TRUE').item() #load the weights again
print(hist1.keys()) # display the parameters of the weights


# summarize history for accuracy
plt.plot(hist1['categorical_accuracy'])
plt.plot(hist1['val_categorical_accuracy'])
plt.title('Model_1 Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist1['loss'])
plt.plot(hist1['val_loss'])
plt.title('Model_1 Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# 2) CUSTOM + ADAM OPT

In [ ]:

model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),



    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),



    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),

    tf.keras.layers.Dense(3, activation='softmax')
])

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])


In [ ]:
visualkeras.layered_view(model2,legend=True)

In [ ]:
model2.summary()

In [ ]:
history2=model2.fit(train_generator, epochs=30, validation_data=validation_generator)

In [ ]:
model2.save('/content/drive/My Drive/hda/yeni_colab/models/model2')

np.save('Weights_model2.npy',history2.history)### history ne burdai

In [ ]:
history2 = keras.models.load_model('/content/drive/My Drive/hda/yeni_colab/models/model2')


In [ ]:
history2.evaluate(train_generator)
history2.evaluate(validation_generator)
history2.evaluate(test_generator)

In [ ]:
from matplotlib import pyplot as plt

hist2=np.load('Weights_model2.npy',allow_pickle='TRUE').item() #load the weights again
print(hist2.keys()) # display the parameters of the weights


# summarize history for accuracy
plt.plot(hist2['categorical_accuracy'])
plt.plot(hist2['val_categorical_accuracy'])
plt.title('Model_2 Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist2['loss'])
plt.plot(hist2['val_loss'])
plt.title('Model_2 Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

#3) ATTENTION LAYER + CUSTOM

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.filters = input_shape[-1]
        self.avg_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.max_pool = tf.keras.layers.GlobalMaxPooling2D()
        self.shared_layer_one = tf.keras.layers.Dense(self.filters, activation='relu')
        self.shared_layer_two = tf.keras.layers.Dense(self.filters, activation='relu')
        self.attention_layer = tf.keras.layers.Dense(self.filters, activation='sigmoid')
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        avg_pool = self.avg_pool(inputs)
        avg_pool = self.shared_layer_one(avg_pool)
        avg_pool = self.shared_layer_two(avg_pool)

        max_pool = self.max_pool(inputs)
        max_pool = tf.keras.layers.Reshape((1, 1, self.filters))(max_pool)
        max_pool = self.shared_layer_one(max_pool)
        max_pool = self.shared_layer_two(max_pool)

        attention = tf.keras.layers.Add()([avg_pool, max_pool])
        attention = self.attention_layer(attention)

        return tf.keras.layers.Multiply()([inputs, attention])

In [ ]:
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),



    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),



    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),

    AttentionLayer(),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model3.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['categorical_accuracy'])

In [ ]:
model3.summary()

In [ ]:
history3=model3.fit(train_generator, epochs=30, validation_data=validation_generator)

Epoch 1/30
43/43 [==============================] - 1151s 26s/step - loss: 1.8301 - categorical_accuracy: 0.6332 - val_loss: 3.5491 - val_categorical_accuracy: 0.3333
Epoch 2/30
43/43 [==============================] - 904s 21s/step - loss: 0.5806 - categorical_accuracy: 0.7843 - val_loss: 6.7618 - val_categorical_accuracy: 0.3333
Epoch 3/30
43/43 [==============================] - 918s 21s/step - loss: 0.4715 - categorical_accuracy: 0.8291 - val_loss: 3.2023 - val_categorical_accuracy: 0.3333
Epoch 4/30
43/43 [==============================] - 907s 21s/step - loss: 0.4219 - categorical_accuracy: 0.8510 - val_loss: 3.3887 - val_categorical_accuracy: 0.3333
Epoch 5/30
43/43 [==============================] - 917s 21s/step - loss: 0.3453 - categorical_accuracy: 0.8780 - val_loss: 2.1864 - val_categorical_accuracy: 0.3333
Epoch 6/30
43/43 [==============================] - 912s 21s/step - loss: 0.3411 - categorical_accuracy: 0.8794 - val_loss: 3.7759 - val_categorical_accuracy: 0.3333
Epo

In [ ]:
visualkeras.layered_view(model3,legend=True)

In [ ]:
model3.save('/content/drive/My Drive/hda/yeni_colab/models/model3')

np.save('/content/drive/My Drive/hda/Weights_model3.npy',history3.history)

In [ ]:
history3 = keras.models.load_model('/content/drive/My Drive/hda/yeni_colab/models/model3')


In [ ]:
history3.evaluate(train_generator)
history3.evaluate(validation_generator)
history3.evaluate(test_generator)

In [ ]:
from matplotlib import pyplot as plt

hist3=np.load('Weights_model3.npy',allow_pickle='TRUE').item() #load the weights again
print(hist3.keys()) # display the parameters of the weights


# summarize history for accuracy
plt.plot(hist3['categorical_accuracy'])
plt.plot(hist3['val_categorical_accuracy'])
plt.title('Model_3 Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist3['loss'])
plt.plot(hist3['val_loss'])
plt.title('Model_3 Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()